In [20]:
import os
import sys

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, PROJECT_ROOT)

PROJECT_ROOT


'c:\\Users\\SARTHAK CHANDEL\\OneDrive\\Desktop\\satellite-property-valuation'

In [2]:
import sys
sys.path.append("..")


In [3]:
import torch
torch.__version__


'2.9.1+cpu'

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor


In [10]:
df = pd.read_excel("../data/raw/train.xlsx")
df = df.dropna()
df.head()


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,9117000170,20150505T000000,268643,4,2.25,1810,9240,2.0,0,0,...,7,1810,0,1961,0,98055,47.4362,-122.187,1660,9240
1,6700390210,20140708T000000,245000,3,2.50,1600,2788,2.0,0,0,...,7,1600,0,1992,0,98031,47.4034,-122.187,1720,3605
2,7212660540,20150115T000000,200000,4,2.50,1720,8638,2.0,0,0,...,8,1720,0,1994,0,98003,47.2704,-122.313,1870,7455
3,8562780200,20150427T000000,352499,2,2.25,1240,705,2.0,0,0,...,7,1150,90,2009,0,98027,47.5321,-122.073,1240,750
4,7760400350,20141205T000000,232000,3,2.00,1280,13356,1.0,0,0,...,7,1280,0,1994,0,98042,47.3715,-122.074,1590,8071


In [11]:
features = [
    "bedrooms",
    "bathrooms",
    "sqft_living",
    "sqft_lot",
    "floors",
    "waterfront",
    "view",
    "condition",
    "grade",
    "sqft_above",
    "sqft_basement",
    "lat",
    "long"
]

X = df[features]
y = df["price"]

X.shape, y.shape


((16209, 13), (16209,))

In [12]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train.shape, X_val.shape


((12967, 13), (3242, 13))

In [21]:
model = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

In [14]:
y_pred = model.predict(X_val)

mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

rmse, r2


(np.float64(126622.50849177445), 0.8722333475657016)

In [15]:
df_mm = df.head(5).copy()

df_mm["image_path"] = [
    "../data/images/train/house_0.png",
    "../data/images/train/house_1.png",
    "../data/images/train/house_2.png",
    "../data/images/train/house_3.png",
    "../data/images/train/house_4.png",
]

df_mm[["price", "lat", "long", "image_path"]]


,price,lat,long,image_path
0,268643,47.4362,-122.187,../data/images/train/house_0.png
1,245000,47.4034,-122.187,../data/images/train/house_1.png
2,200000,47.2704,-122.313,../data/images/train/house_2.png
3,352499,47.5321,-122.073,../data/images/train/house_3.png
4,232000,47.3715,-122.074,../data/images/train/house_4.png


In [22]:
from src.dataset import HouseDataset


In [18]:
from src.dataset import HouseDataset

feature_cols = [
    "bedrooms",
    "bathrooms",
    "sqft_living",
    "sqft_lot",
    "floors",
    "waterfront",
    "view",
    "condition",
    "grade",
    "sqft_above",
    "sqft_basement",
    "lat",
    "long"
]

dataset = HouseDataset(df_mm, feature_cols)

tab, img, price = dataset[0]

tab.shape, img.shape, price


(torch.Size([13]), torch.Size([3, 256, 256]), tensor(268643.))

In [19]:
dataset = HouseDataset(df_mm, feature_cols)
tab, img, price = dataset[0]
tab.shape, img.shape, price


(torch.Size([13]), torch.Size([3, 256, 256]), tensor(268643.))

In [ ]:
from src.model import MultimodalNet

In [24]:
tabular_dim = tab.shape[0]
model_mm = MultimodalNet(tabular_dim)

model_mm

MultimodalNet(
  (tabular_net): TabularNet(
    (fc1): Linear(in_features=13, out_features=64, bias=True)
    (fc2): Linear(in_features=64, out_features=32, bias=True)
  )
  (image_net): ImageNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc): Linear(in_features=131072, out_features=64, bias=True)
  )
  (fc1): Linear(in_features=96, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [25]:
tab_batch = tab.unsqueeze(0)
img_batch = img.unsqueeze(0)

output = model_mm(tab_batch, img_batch)

output.shape, output


(torch.Size([1, 1]), tensor([[-64.8236]], grad_fn=<AddmmBackward0>))

In [26]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=2, shuffle=True)


In [27]:
import torch.optim as optim

criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model_mm.parameters(), lr=0.001)


In [29]:
model_mm.train()

for epoch in range(5):
    epoch_loss = 0.0

    for tab_b, img_b, price_b in loader:
        optimizer.zero_grad()

        preds = model_mm(tab_b, img_b).view(-1)
        loss = criterion(preds, price_b)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.2f}")


Epoch 1, Loss: 200450473984.00
Epoch 2, Loss: 234515296256.00
Epoch 3, Loss: 232600666112.00
Epoch 4, Loss: 197448019968.00
Epoch 5, Loss: 224672505856.00


In [32]:
df_test = pd.read_excel("../data/raw/test.xlsx")
df_test = df_test.dropna()
df_test.head()


,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2591820310,20141006T000000,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700
1,7974200820,20140821T000000,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283
2,7701450110,20140815T000000,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685
3,9522300010,20150331T000000,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226
4,9510861140,20140714T000000,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050


In [33]:
X_test = df_test[features]
X_test.shape


(5404, 13)

In [34]:
test_preds = model.predict(X_test)
test_preds[:5]


array([ 353413.5 ,  964537.9 , 1036961.9 , 1925885.8 ,  713967.44])

In [35]:
submission = pd.DataFrame({
    "id": df_test["id"],
    "predicted_price": test_preds
})

submission.head()


,id,predicted_price
0,2591820310,353413.50
1,7974200820,964537.90
2,7701450110,1036961.90
3,9522300010,1925885.80
4,9510861140,713967.44


In [36]:
submission_path = "../outputs/predictions.csv"
submission.to_csv(submission_path, index=False)

submission_path


'../outputs/predictions.csv'